# --- Day 18: RAM Run ---
https://adventofcode.com/2024/day/18

In [1]:
def getCorruptedBytes():
    with open("corruptedBytes.txt") as file:
        return file.read()

In [2]:
# Formatting
byteCoords = getCorruptedBytes()
byteCoords = [list(map(int, coords.split(","))) for coords in byteCoords.split("\n")]
byteCoords = byteCoords[:1024] # Get only the first 1024 corrupted byte coordinates

# Create memory grid ("." = safe byte | "#" = corrupt byte)
gridsize = 70
startLoc = (0,0)
endLoc = (gridsize, gridsize)
memory = [["."] * (gridsize + 1) for _ in range(gridsize + 1)]
for x, y in byteCoords: # Mark corrupted bytes
    memory[y][x] = "#"

# Node class used for easily backtracking to find the path from where it started
class Node:
    def __init__(self, coords: tuple[int], parent=None):
        self.coords = coords
        self.parent = parent

    def __eq__(self, value):
        # If both values are nodes
        if type(self) == type(value):
            return self.coords == value.coords
        # Otherwise one value should be a tuple (coordinates)
        return self.coords == value
    
    # Get the coordinates from all the previous nodes
    @property
    def visitedCoords(self) -> list[tuple[int]]:
        allCoords = []
        currentNode = self

        # Continue until the parent is None
        while currentNode.parent:
            allCoords.append(currentNode.coords)
            currentNode = currentNode.parent
        # Add the last (really the first) coordinate to the list
        allCoords.append(currentNode.coords)

        return allCoords

# Returns True if coordinates are in bounds, otherwise False
def inBounds(memory: list[list[str]], coords) -> bool:
    rowInBounds = coords[0] >= 0 and coords[0] < len(memory)
    colInBounds = coords[1] >= 0 and coords[1] < len(memory[0])
    return rowInBounds and colInBounds

# Find shortest path using breadth first search!
def findPath(memory: list[list[str]], startCoords: tuple[int], endCoords: tuple[int]) -> list[tuple[int]]:
    # Create the frontier and explored list
    startNode = Node(coords=startCoords)
    frontier = [startNode]
    explored = [startNode]

    # Loop until the frontier is empty
    while frontier:
        currentNode = frontier.pop(0)
        currentCoords = currentNode.coords

        # If we've made it to the end, just return that node
        if currentNode == endCoords:
            return currentNode

        # Get surrounding coordinates
        up = (currentCoords[0] - 1, currentCoords[1])
        down = (currentCoords[0] + 1, currentCoords[1])
        left = (currentCoords[0], currentCoords[1] - 1)
        right = (currentCoords[0], currentCoords[1] + 1)

        # Loop through each direction
        for direction in [up, down, left, right]:
            # If it's within the bounds of the memory chunk
            if inBounds(memory, direction):
                # If the byte has not already been explored add it to the frontier and mark it as explored
                if direction not in explored and memory[direction[0]][direction[1]] != "#":
                    frontier.append(Node(direction, currentNode))
                    explored.append(direction)

shortestPath = findPath(memory, startLoc, endLoc)
print(f"Minimum number of steps to reach the exit: {len(shortestPath.visitedCoords) - 1}")

Minimum number of steps to reach the exit: 344


# --- Part Two ---

In [22]:
from heapq import heappush, heappop

# Formatting
byteCoords = getCorruptedBytes()
byteCoords = [list(map(int, coords.split(","))) for coords in byteCoords.split("\n")]

# Create memory grid
gridsize = 70
startLoc = (0,0)
endLoc = (gridsize, gridsize)
memory = [["."] * (gridsize + 1) for _ in range(gridsize + 1)]

# Node class used for easily backtracking to find the path from where it started
class Node:
    def __init__(self, coords: tuple[int], endCoords: tuple[int], parent=None):
        self.coords = coords
        self.endCoords = endCoords
        self.parent = parent

    def __eq__(self, value):
        # If both values are nodes
        if type(self) == type(value):
            return self.coords == value.coords
        # Otherwise one value should be a tuple (coordinates)
        return self.coords == value
    
    # Get the coordinates from all the previous nodes
    @property
    def visitedCoords(self) -> list[tuple[int]]:
        allCoords = []
        currentNode = self

        # Continue until the parent is None
        while currentNode.parent:
            allCoords.append(currentNode.coords)
            currentNode = currentNode.parent
        # Add the last (really the first) coordinate to the list
        allCoords.append(currentNode.coords)

        return allCoords
    
    # Returns True if the cost of the object is less than the cost of the object passed in
    def __lt__(self, other):
        return self.cost < other.cost
    
    # Return the cost of the move using manhattan distance as a heuristic
    @property
    def cost(self):
        return (1 + abs(self.coords[0] - self.endCoords[0]) + abs(self.coords[1] - self.endCoords[1]))

# Returns True if coordinates are in bounds, otherwise False
def inBounds(memory: list[list[str]], coords) -> bool:
    rowInBounds = coords[0] >= 0 and coords[0] < len(memory)
    colInBounds = coords[1] >= 0 and coords[1] < len(memory[0])
    return rowInBounds and colInBounds

# Find shortest path using A*!
def astar(memory: list[list[str]], startCoords: tuple[int], endCoords: tuple[int]) -> list[tuple[int]]:
    # Create the frontier and explored list
    startNode = Node(coords=startCoords, endCoords=endCoords)
    frontier = []
    heappush(frontier, startNode) # Push the start node to the queue
    explored = [startNode]

    # Loop until the frontier is empty
    while frontier:
        currentNode = heappop(frontier)
        currentCoords = currentNode.coords

        # If we've made it to the end, just return that node
        if currentNode == endCoords:
            return currentNode

        # Get surrounding coordinates
        up = (currentCoords[0] - 1, currentCoords[1])
        down = (currentCoords[0] + 1, currentCoords[1])
        left = (currentCoords[0], currentCoords[1] - 1)
        right = (currentCoords[0], currentCoords[1] + 1)

        # Loop through each direction
        for direction in [up, down, left, right]:
            # If it's within the bounds of the memory chunk
            if inBounds(memory, direction):
                # If the byte has not already been explored add it to the frontier and mark it as explored
                if direction not in explored and memory[direction[0]][direction[1]] != "#":
                    heappush(frontier, Node(direction, endCoords, currentNode))
                    explored.append(direction)

    # If we make it through the search without finding our goal, then there is no path to the end
    return None

# Loop through each byte that falls and mark it as corrupted
for x, y in byteCoords:
    memory[y][x] = "#"

    # If the newest corrupted byte is not on what is already the shortest path, don't bother searching again!
    if [x,y] != byteCoords[0] and (y,x) not in shortestPath.visitedCoords:
        continue

    # If there does not exist a path, then that is the first byte that blocks the exit
    shortestPath = astar(memory, startLoc, endLoc)
    if shortestPath == None:
        print(f"First byte to prevent the exit from being reachable: {x},{y}")
        break

First byte to prevent the exit from being reachable: 46,18


## Optimizations
- Use A* instead of breadth first search
- Start searching at 1024 since we know from part 1 it's not before that
- If the newest byte does not fall on a coordinate in the path, don't re-search, just continue
- Do a binary search for the right number of falling bytes